### Pattern finding functions

In [ ]:
def find_pattern_first_index(bitstream):
    pattern = "3c5c"
    pattern_length = len(pattern) * 4  # 4 bits per hexadecimal digit

    for i in range(len(bitstream) - pattern_length + 1):
        current_substring = bitstream[i:i+pattern_length]
        hexadecimal = hex(int(current_substring, 2))[2:]  # Convert binary to hexadecimal
        if hexadecimal == pattern:
            return i

    return -1  # Pattern not found

def find_pattern_multiple_indexs(bitstream):
    pattern = "3c5c"
    pattern_length = len(pattern) * 4  # 4 bits per hexadecimal digit

    indices = []  # List to store the indices of pattern occurrences

    for i in range(len(bitstream) - pattern_length + 1):
        current_substring = bitstream[i:i+pattern_length]
        hexadecimal = hex(int(current_substring, 2))[2:]  # Convert binary to hexadecimal
        if hexadecimal == pattern:
            indices.append(i)

    return indices  # Return the list of indices

### Translate binary to readable data

In [ ]:
def translate_with_indices(input_stream, positions, parent_dir, output, chipID):
    f = open(parent_dir+'/'+output, 'w')
    residual = ''
    
    # hex chipID to binary
    binID = format(int(chipID, 0), '017b')
    wordlength = 40
    printline = ''
    key = ''
    
    for i, index in enumerate(positions):

        word = input_stream[index:index+wordlength]
        printline = "ETROC2 0 "# + "{:d} ".format(channel)

        if len(word) != 40:
            residual = word
            break
        elif len(word) == 40:
            # Header
            if word[0:16] == '0011110001011100' and word[16:18] == '00':
                printline += "HEADER "
                printline += "L1COUNTER " + word[18:26] + " "
                printline += "TYPE " + word[26:28] + " "
                printline += "BCID " + f"{int(word[28:40], base=2)}" + "\n"
                key = 'header'
            # Frame filler
            elif word[0:16] == '0011110001011100' and word[16:18] == '10':
                printline += "FRAMEFILLER "
                printline += "L1COUNTER " + word[18:26] + " "
                printline += "EBS " + word[26:28] + " "
                printline += "BCID " + f"{int(word[28:40], base=2)}" + "\n"
                key = 'filler'
            # Firmware filler
            elif word[0:16] == '0011110001011100' and word[16:18] == '11':
                printline += "FIRMWAREFILLER "
                printline += "MISSINGCOUNT " + word[18:40] + "\n"
                key = 'filler'
            else:
                printline += "NOT DEFINED " + word[0:16] + " " + word[16:18] + " " + word[18:] + "\n"
                key = 'whatisthis'
                pass
        
        # Save if the data is filler
        if key == 'filler':
            f.write(printline)
        
        # if the data is header, try to find the data and trailer 
        elif key == 'header':
            try: 
                loop = int((positions[i+1] - positions[i])/40)
                for k in range(1, loop):
                    word = input_stream[index+(wordlength*k):index+(wordlength*(k+1))]
                    # Trailer
                    if word[0:18] == '0'+str(binID):
                        printline += "TRAILER "
                        printline += "CHIPID " + f"{hex(int(word[1:18], base=2))}" + " "
                        printline += "STATUS " + word[18:24] + " "
                        printline += "HITS " + f"{int(word[24:32], base=2)}" + " "
                        printline += "CRC " + word[32:40] + "\n"
                        key = 'trailer'
                    
                    elif word[0] == '1':
                        printline += "DATA "
                        printline += "EA " + word[1:3] + " "
                        printline += "COL " + "{:d} ".format(int(word[3:7], base=2))
                        printline += "ROW " + "{:d} ".format(int(word[7:11], base=2))
                        printline += "TOA " + "{:d} ".format(int(word[11:21], base=2))
                        printline += "TOT " + "{:d} ".format(int(word[21:30], base=2))
                        printline += "CAL " + "{:d} ".format(int(word[30:40], base=2)) + "\n"
                        key = 'data'
                    
                    # if the data is trailer, write the output in the file
                    if key == 'trailer':
                        f.write(printline)
            except:
                # out of index range, move to the next file
                residual = word

    f.close()
    return residual        

In [ ]:
# install natsort module (skip if this module is installed already!)
import sys
!{sys.executable} -m pip install natsort

### Let's convert!

In [ ]:
from glob import glob
from natsort import natsorted, ns
from tqdm import tqdm
from pathlib import Path

# natural sorting in python
dir=Path('/home/jongho/Physics/ETROC/etroc2_translate/test9')
files = glob(str(dir)+'/*Data_[0-9]*.dat')
files = natsorted(files)

# print the list of files to check 
# for file in files: print(file)

In [ ]:
residual = ''

import os
if not os.path.exists(str(dir/'StandaloneTranslate')):
    os.mkdir(str(dir/'StandaloneTranslate'))

for i, ifile in enumerate(tqdm(files)):
    # if i > 2: break
    # Let's make a very long bitstream single line
    bitstream = residual + ''
    with open(ifile, 'r') as infile:
        # make data bitstream in a single line
        for line in infile.readlines():
            if line[0:4] == '1100':
                bitstream += line.strip()[4:]

    positions = find_pattern_multiple_indexs(bitstream)
    outname = 'TDC_Data_translated_'+str(i)+'.dat'
    residual = translate_with_indices(bitstream, positions, str(dir/'StandaloneTranslate'), outname, '0x17f0f')